In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gpt2/keras/gpt2_base_en/2/config.json
/kaggle/input/gpt2/keras/gpt2_base_en/2/tokenizer.json
/kaggle/input/gpt2/keras/gpt2_base_en/2/metadata.json
/kaggle/input/gpt2/keras/gpt2_base_en/2/model.weights.h5
/kaggle/input/gpt2/keras/gpt2_base_en/2/assets/tokenizer/merges.txt
/kaggle/input/gpt2/keras/gpt2_base_en/2/assets/tokenizer/vocabulary.json
/kaggle/input/taylorswiftlyrics/taylor_swift_lyrics.csv


In [2]:
os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
import keras_hub
import tensorflow as tf
import tensorflow.data as tf_data

In [3]:
from keras_hub.models import GPT2CausalLMPreprocessor, GPT2CausalLM
from keras_hub.samplers import TopPSampler

In [4]:
Preprocessor = GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length = 512,
)

In [24]:
Sampler = TopPSampler(
    p = 0.84,
    k = None,
    seed = None,
    temperature = 0.89
)

In [6]:
GPT2Model = GPT2CausalLM.from_preset(
    "gpt2_base_en",
    preprocessor=Preprocessor
)

We will fine-tune this LLM to generate Taylor Swift-esque lyrics

In [7]:
import re
remove_tags = lambda text: re.sub(r'\[.*?\]', '', text)

no_special_tokens = False

In [8]:
lyrics = pd.read_csv('/kaggle/input/taylorswiftlyrics/taylor_swift_lyrics.csv').dropna()

In [9]:
if no_special_tokens:
    lyrics["Lyrics"] = lyrics["Lyrics"].apply(remove_tags)

In [10]:
dataset = tf_data.Dataset.from_tensor_slices(lyrics["Lyrics"].values)

In [11]:
dataset = dataset.batch(16).cache().prefetch(tf_data.AUTOTUNE)

In [12]:
from keras.optimizers import Adam
from keras.optimizers.schedules import PolynomialDecay
from keras.losses import SparseCategoricalCrossentropy

In [13]:
scheduler = PolynomialDecay(
    5e-5,
    decay_steps=dataset.cardinality() * 10,
    end_learning_rate=0.0,
)

In [14]:
loss = SparseCategoricalCrossentropy(from_logits=True)

In [25]:
GPT2Model.compile(
    optimizer=Adam(scheduler),
    loss=loss,
    weighted_metrics=["accuracy"],
    sampler=Sampler,
)

In [26]:
GPT2Model.summary()

Preprocessor: "gpt2_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gpt2_tokenizer (GPT2Tokenizer)                                │                       Vocab size: 50,257 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gpt2_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gpt2_backbone (GPT2Backbone)  │ (None, None, 768)         │     124,439,808 │ padding_mask[0][0],        │
│                               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 50257)       │      38,597,376 │ gpt2_backbone[0][0]        │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 124,439,808 (474.70 MB)

 Trainable params: 124,439,808 (474.70 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
GPT2Model.fit(dataset, epochs=30)

Epoch 1/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 130s 6s/step - accuracy: 0.5931 - loss: 1.8764
Epoch 2/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 34s 3s/step - accuracy: 0.6030 - loss: 1.8005
Epoch 3/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.6093 - loss: 1.7597
Epoch 4/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.6156 - loss: 1.7290
Epoch 5/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.6188 - loss: 1.7066
Epoch 6/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.6236 - loss: 1.6823
Epoch 7/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.6256 - loss: 1.6627
Epoch 8/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.6272 - loss: 1.6515
Epoch 9/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.6290 - loss: 1.6426
Epoch 10/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.6309 - loss: 1.6328
Epoch 11/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.6303 - loss: 1.6322
Epoch 12/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.6293 - l

In [28]:
test = GPT2Model.generate(
    "And I said, Romeo take me",
    max_length = 116,
)

print(test)

And I said, Romeo take me home
And I'll be fine, babe
I've been taking so long
Oh, oh, oh
I've been so busy
I've been so busy
Oh, oh, oh
Oh, oh, oh
[Verse 1]
I was doing all the time, babe
You're the one who says
I'll never, ever, ever do
It's been so long
It's been so long
[Pre-Chorus]
So what's the last time?
Oh, oh
